Data fields:

* Id - an Id that represents a (Store, Date) duple within the test set
* Store - a unique Id for each store
* Sales - the turnover for any given day (this is what you are predicting)
* Customers - the number of customers on a given day
* Open - an indicator for whether the store was open: 0 = closed, 1 = open
* StateHoliday - indicates a state holiday. Normally all stores, with few exceptions, are closed on state holidays. Note that all schools are closed on public holidays and weekends. a = public holiday, b = Easter holiday, c = Christmas, 0 = None
* SchoolHoliday - indicates if the (Store, Date) was affected by the closure of public schools
* StoreType - differentiates between 4 different store models: a, b, c, d
* Assortment - describes an assortment level: a = basic, b = extra, c = extended
* CompetitionDistance - distance in meters to the nearest competitor store
* CompetitionOpenSince[Month/Year] - gives the approximate year and month of the time the nearest competitor was opened
* Promo - indicates whether a store is running a promo on that day
* Promo2 - Promo2 is a continuing and consecutive promotion for some stores: 0 = store is not participating, 1 = store is participating
* Promo2Since[Year/Week] - describes the year and calendar week when the store started participating in Promo2
* PromoInterval - describes the consecutive intervals Promo2 is started, naming the months the promotion is started anew. E.g. "Feb,May,Aug,Nov" means each round starts in February, May, August, November of any given year for that store

# Create the following competition/promotion variables

 * CompetitionOn - There is currently competition for store x on day y
 
From https://www.kaggle.com/c/rossmann-store-sales/discussion/16867#97369:
"Promo2 is a coupon based mailing campaign that is send to customers of participating stores.
Each letter contains several coupons, mostly for a general discount on all products which are valid for three months.
So after/right before these coupons have expired we send a new round of mail to our customers"

Create 2 columns:
 * MonthsSinceCouponsMailed (use sentinel value if promo is not currently operating)

In [1]:
%pylab inline

#from __future__ import division

import timeit

# Data handling
import numpy as np
import pandas as pd

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns

print pd.__version__

Populating the interactive namespace from numpy and matplotlib
0.24.1


# Data cleaning functions

In [2]:
def clean_competition_distance(df):
    """If the CompetitionDistance is NaN, then there isn't any competition.
    Just replace the NaN with the maximum competition distance.
    """
    max_distance = df['CompetitionDistance'].max()
    df['CompetitionDistance'] = df['CompetitionDistance'].fillna(value=max_distance)
    return df

In [3]:
def clean_competition_open_since_date(df):
    """If the CompetitionOpenSinceYear and CompetitionOpenSinceMonth are NaN, 
    then the competition was always open. 
    So, replace the NaN with the earliest possible year and month.
    """
    min_year = df['CompetitionOpenSinceYear'].min()
    min_month = 1
    df['CompetitionOpenSinceYear'] = df['CompetitionOpenSinceYear'].fillna(value=min_year)
    df['CompetitionOpenSinceMonth'] = df['CompetitionOpenSinceMonth'].fillna(value=min_month)
    return df

In [4]:
def convert_date_to_ymd(df):
    """Extract year, month, day from 'Date' column,
    then replace it with the 3 columns 'Year', 'Month', 'DayOfMonth'. 
    """
    year = df['Date'].apply(lambda x: int(x[:4]))
    month = df['Date'].apply(lambda x: int(x[5:7]))
    day = df['Date'].apply(lambda x: int(x[8:]))
    
    # Create new columns
    df['Year'] = year
    df['Month'] = month
    df['DayOfMonth'] = day
    return df

In [5]:
import datetime

def month_of_year_from_week_of_year(year, week):
    """Convert week of year to approximate month of year.
    You don't know the exact day, so this will be approximate.
    
    Parameters
    ----------
    year : int
    week : int (1 to 52)
    
    Returns
    -------
    month : int (1 to 12)
    """
    # Days from the first day of the year
    day = (week-1)*7
    # datetime indexes days and months starting at 1
    date = datetime.datetime(int(year), 1, 1) + datetime.timedelta(int(day))
    # You want months indexed starting at 1
    return date.month

In [6]:
def promo_interval_to_months_since_coupons_mailed(month, promo_interval):
    """Determine the months since coupons were mailed.
    If coupons were never mailed, use a sentinel value of 3
    
    Parameters
    ----------
    month : int 
        Month number ('Jan'=1, 'Dec'=12)
    promo_interval : str
        [nan, 'Jan,Apr,Jul,Oct', 'Feb,May,Aug,Nov', 'Mar,Jun,Sept,Dec']
        
    Returns
    -------
    months_since : int
        Months since the coupons were mailed
    """
    if promo_interval=='Jan,Apr,Jul,Oct':
        months_since = [0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2]
        return months_since[month-1]
    elif promo_interval=='Feb,May,Aug,Nov':
        months_since = [2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1]
        return months_since[month-1]
    elif promo_interval=='Mar,Jun,Sept,Dec':
        months_since = [1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0]
        return months_since[month-1]
    else:
        return -1

In [7]:
def clean_store_data(df_store):
    df_store = clean_competition_distance(df_store)
    df_store = clean_competition_open_since_date(df_store)
    return df_store

# Load data

In [8]:
# More detailed data about each store
df_store = pd.read_csv('store.csv', low_memory=False)

# Data for each store for each day. Nrows ~ Nstores*Ndays
df_train = pd.read_csv('train.csv', low_memory=False)
df_test = pd.read_csv('test.csv', low_memory=False)

# Clean Store data

In [9]:
df_store = clean_store_data(df_store)

# Join train/test sets with store data, then clean it

In [10]:
def months_since_coupons_mailed(df_row):
    """If there is a coupon promotion, give the months since the coupons were mailed.
    Otherwise, return -1.
    """
    year = df_row['Year']
    month = df_row['Month']
    promo_start_year = df_row['Promo2SinceYear']
    promo_start_week = df_row['Promo2SinceWeek']
    promo_interval = df_row['PromoInterval']
    #print year, month, promo_start_year, promo_start_week, promo_interval
    
    # Test if there ever was a promo
    if (np.isnan(promo_start_year) | np.isnan(promo_start_week)):
        return -1
       
    # Test if the promo has started
    promo_start_month = month_of_year_from_week_of_year(promo_start_year, promo_start_week)
    #print promo_start_month
    if (12*year+month)>=(12*promo_start_year+promo_start_month):
        return promo_interval_to_months_since_coupons_mailed(month, promo_interval)
    else:
        return -1

In [11]:
def competition_open_flag(df_row):
    """Flag if a competitor's store is open.
    """
    year = df_row['Year']
    month = df_row['Month']
    competition_open_year = df_row['CompetitionOpenSinceYear']
    competition_open_month = df_row['CompetitionOpenSinceMonth']
    
    if (12*year+month)>=(12*competition_open_year+competition_open_month):
        return 1
    else:
        return 0

In [12]:
# Pandas df.merge() performs a database-style join. Don't use the Pandas df.join() function.
df_train = df_train.merge(df_store, on='Store', how='inner')
df_test = df_test.merge(df_store, on='Store', how='inner')

In [13]:
# TODO: You could switch this to a 6 week trailing moving average instead of a global average

# Get the average daily sales for a given store for the training set
store_id = df_train['Store'].unique()
for i in store_id:
    mean_sales = df_train[df_train['Store']==i]['Sales'].mean()
    df_train.loc[(df_train['Store']==i), 'MeanSales'] = mean_sales
    
# Now copy the training set value over to the test set.
# You won't know the test-set value in advance (it's one of the things you're trying to predict).
store_id = df_test['Store'].unique()
for i in store_id:
    mean_sales = df_train[df_train['Store']==i]['MeanSales'].mean()
    df_test.loc[(df_test['Store']==i), 'MeanSales'] = mean_sales

In [14]:
# Fix the 11 'NaN' values in the 'Open' column of the test set.
# Better to assume store 622 is open than closed.
df_test['Open'] = df_test['Open'].fillna(value=1)

In [15]:
# Make columns for 'Year', 'Month', 'DayOfMonth'
df_train = convert_date_to_ymd(df_train)
df_test = convert_date_to_ymd(df_test)

In [16]:
# Make column for months since last coupon booklet was mailed

t0 = timeit.time.time()

months_since_coupons = [months_since_coupons_mailed(df_train.loc[i]) for i in range(len(df_train))]
df_train['MonthsSinceCoupons'] = months_since_coupons

months_since_coupons = [months_since_coupons_mailed(df_test.loc[i]) for i in range(len(df_test))]
df_test['MonthsSinceCoupons'] = months_since_coupons

t1 = timeit.time.time()
print t1-t0

399.349657059


In [17]:
# Make binary column for a competitor's store nearby

t0 = timeit.time.time()

competition_open = [competition_open_flag(df_train.loc[i]) for i in range(len(df_train))]
df_train['CompetitionOpen'] = competition_open

competition_open = [competition_open_flag(df_test.loc[i]) for i in range(len(df_test))]
df_test['CompetitionOpen'] = competition_open

t1 = timeit.time.time()
print t1-t0

384.17772603


In [18]:
df_test.head()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,...,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,MeanSales,Year,Month,DayOfMonth,MonthsSinceCoupons,CompetitionOpen
0,1,1,4,2015-09-17,1.0,1,0,0,c,a,...,0,NaN,NaN,NaN,3945.704883,2015,9,17,-1,1
1,857,1,3,2015-09-16,1.0,1,0,0,c,a,...,0,NaN,NaN,NaN,3945.704883,2015,9,16,-1,1
2,1713,1,2,2015-09-15,1.0,1,0,0,c,a,...,0,NaN,NaN,NaN,3945.704883,2015,9,15,-1,1
3,2569,1,1,2015-09-14,1.0,1,0,0,c,a,...,0,NaN,NaN,NaN,3945.704883,2015,9,14,-1,1
4,3425,1,7,2015-09-13,0.0,0,0,0,c,a,...,0,NaN,NaN,NaN,3945.704883,2015,9,13,-1,1


In [19]:
# Do one-hot-encoding for the 3 categorical columns
df_train = pd.get_dummies(df_train, columns=['StoreType', 'Assortment', 'StateHoliday'])
df_test = pd.get_dummies(df_test, columns=['StoreType', 'Assortment', 'StateHoliday'])

In [22]:
print df_train.columns
print df_test.columns

Index([u'Store', u'DayOfWeek', u'Date', u'Sales', u'Customers', u'Open',
       u'Promo', u'SchoolHoliday', u'CompetitionDistance',
       u'CompetitionOpenSinceMonth', u'CompetitionOpenSinceYear', u'Promo2',
       u'Promo2SinceWeek', u'Promo2SinceYear', u'PromoInterval', u'MeanSales',
       u'Year', u'Month', u'DayOfMonth', u'MonthsSinceCoupons',
       u'CompetitionOpen', u'StoreType_a', u'StoreType_b', u'StoreType_c',
       u'StoreType_d', u'Assortment_a', u'Assortment_b', u'Assortment_c',
       u'StateHoliday_0', u'StateHoliday_a', u'StateHoliday_b',
       u'StateHoliday_c'],
      dtype='object')
Index([u'Id', u'Store', u'DayOfWeek', u'Date', u'Open', u'Promo',
       u'SchoolHoliday', u'CompetitionDistance', u'CompetitionOpenSinceMonth',
       u'CompetitionOpenSinceYear', u'Promo2', u'Promo2SinceWeek',
       u'Promo2SinceYear', u'PromoInterval', u'MeanSales', u'Year', u'Month',
       u'DayOfMonth', u'MonthsSinceCoupons', u'CompetitionOpen',
       u'StoreType_a', u'StoreT

In [24]:
df_train.columns.difference(df_test.columns)

Index([u'Customers', u'Sales', u'StateHoliday_b', u'StateHoliday_c'], dtype='object')

In [27]:
def add_missing_dummy_columns(df, columns):
    """Create a column of 0's if it does not already exist
    """
    missing_cols = set(columns) - set(df.columns)
    for c in missing_cols:
        print c
        df[c] = 0

In [28]:
# The only StateHoliday in the test set is type a
# a = public holiday, b = Easter holiday, c = Christmas
# Test set is August, September
# August 15: "Assumption of Mary" in Saarland and Bavaria
add_missing_dummy_columns(df_test, ['StateHoliday_0', 'StateHoliday_a', 'StateHoliday_b', 'StateHoliday_c'])

StateHoliday_b
StateHoliday_c


In [34]:
df_test['StateHoliday_a'].describe()

count    41088.000000
mean         0.004381
std          0.066044
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: StateHoliday_a, dtype: float64

In [36]:
print df_train.isna().sum()
print df_test.isna().sum()

Store                             0
DayOfWeek                         0
Date                              0
Sales                             0
Customers                         0
Open                              0
Promo                             0
SchoolHoliday                     0
CompetitionDistance               0
CompetitionOpenSinceMonth         0
CompetitionOpenSinceYear          0
Promo2                            0
Promo2SinceWeek              508031
Promo2SinceYear              508031
PromoInterval                508031
MeanSales                         0
Year                              0
Month                             0
DayOfMonth                        0
MonthsSinceCoupons                0
CompetitionOpen                   0
StoreType_a                       0
StoreType_b                       0
StoreType_c                       0
StoreType_d                       0
Assortment_a                      0
Assortment_b                      0
Assortment_c                

# Save the cleaned data

In [37]:
df_train.to_csv('train_clean.csv', index=False)
df_test.to_csv('test_clean.csv', index=False)